## Create SQL table

In [ ]:
import pandas as pd
import sqlalchemy
from utils_sql import get_connection, get_thresholds_df, update_create_table, query_metrics_table
from sqlalchemy import text, Table, Column, String, Integer, Float, MetaData, PrimaryKeyConstraint

img_benchmarks = "image_nvai_benchmarks"
img_metrics = "image_nvai_metrics"
vid_benchmarks = "video_nvai_benchmarks"
vid_metrics = "video_nvai_metrics"

FILES_PATH = "/Users/irinakw/Library/CloudStorage/GoogleDrive-i.white@neuronsinc.com/Shared drives/HQ - R&D/Benchmark Documents/sql_tables_files/"

data_img_benchmarks_path = FILES_PATH + 'img_benchmarks_jan2025.csv'
data_img_metrics_path = FILES_PATH + 'img_metrics_jan2025.csv'

data_vid_benchmarks_path = FILES_PATH + 'vid_benchmarks_jan2025.csv'
data_vid_metrics_path = FILES_PATH + 'vid_metrics_jan2025.csv'

In [ ]:
# Create SQLAlchemy engine
engine = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=get_connection
)


In [ ]:
# Create dinamic dropdown values
def get_dropdown_values(table_name, column_name):
    query = f"SELECT DISTINCT {column_name} FROM {table_name}"
    with engine.connect() as connection:
        result = connection.execute(query)
        result = result.fetchall()
    return [r[0] for r in result]

# given 7 columns, when one value of the column is changed, the other columns are updated
initial_values = {
        "industry_category": ['all'],
        "industry_subcategory": ['all'],
        "usecase_category": ['all'],
        "usecase_subcategory": ['all'],
        "platform": ['all', 'facebook_ads', 'instagram_ads', 'not_applicable', 'twitter_ads'],
        "device": ['all', 'desktop', 'mobile', 'not_applicable'],
        "context": ['all', 'no', 'yes'],
    },

### DATABASE QUERY

In [ ]:
filters = {
    "industry_category": "all",
    "industry_subcategory": "all",
    "usecase_category": "all",
    "usecase_subcategory": "all",
    "platform": "youtube_ads",
    "device": "all",
    "context": "no",
    "metric": ["focus", "engagement_frt", "memory","cognitive_demand"],
    "time": "total"
}

In [ ]:
df=query_metrics_table(engine, "Image", **filters)
threshold_df = get_thresholds_df(engine, "Image", get_filters=filters)
THRESHOLDS = threshold_df.set_index("metric")["threshold"].to_dict()
THRESHOLDS


In [ ]:
threshold_df

# UPDATE DATA in SQL DATABASES

In [ ]:
mutual_key_columns = [
            "industry_category", "industry_subcategory", "usecase_category",
            "usecase_subcategory", "platform", "device", "context",
            "metric", "time"
        ]
benchmarks_key_columns = mutual_key_columns + ["type"]
metrics_key_columns = mutual_key_columns + ["asset_id"]

update_create_table(engine, table_name=vid_benchmarks, csv_file_path = data_vid_benchmarks_path, primary_key_columns = benchmarks_key_columns)
update_create_table(engine, table_name=vid_metrics, csv_file_path = data_vid_metrics_path, primary_key_columns = metrics_key_columns)
update_create_table(engine, table_name=img_benchmarks, csv_file_path = data_img_benchmarks_path, primary_key_columns = benchmarks_key_columns)  
update_create_table(engine, table_name=img_metrics, csv_file_path = data_img_metrics_path, primary_key_columns = metrics_key_columns)

In [ ]:
####### BASH CODE TO COPY CSV DATA FROM LOCAL TO CLOUD SQL #######
##################################################################

# % gcloud auth print-access-token  
# use the above for password  below
# % psql -h 127.0.0.1 -U i.white@neuronsinc.com -d assets-experiment

# assets-experiment=> SELECT current_user, current_database();
#       current_user      | current_database  
# ------------------------+-------------------
#  i.white@neuronsinc.com | assets-experiment
# (1 row)

# assets-experiment=> SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';

# assets-experiment=> SELECT COUNT(*) FROM public.data_vid_benchmarks;

# assets-experiment=> GRANT INSERT ON TABLE public.data_vid_benchmarks TO "i.white@neuronsinc.com";

# assets-experiment=> \copy public.video_nvai_metrics (
#     asset_id,industry_category,industry_subcategory,usecase_category,usecase_subcategory,platform,device,context,path_bucket,metric,value,time
# ) 
# FROM 'sql_table/vid_metrics_jan2025_missing_rows.csv' 
# DELIMITER ',' 
# CSV HEADER;


#### ERROR
# psql: error: connection to server at "127.0.0.1", port 5432 failed: Connection refused
#         Is the server running on that host and accepting TCP/IP connections?

# cloud_sql_proxy -instances=neurons-development:us-central1:nh-staging-db-instance=tcp:5432
# % lsof -i :5432  (open new terminal)


### IMAGES

In [ ]:
filters = {
    "industry_category": "all",
    "industry_subcategory": "all",
    "usecase_category": "all",
    "usecase_subcategory": "all",
    "platform": "youtube_ads",
    "device": "all",
    "context": "no",
    "metric": ["focus", "engagement_frt", "memory","cognitive_demand"],
    "time": "total"
}

thresholds_df = get_thresholds_df(engine, 'Video', get_filters=filters)
thresholds_df

In [ ]:
query_metrics_table(engine, 'Video', **filters)


In [ ]:
def run_query(engine, query):
    """
    Query the metrics table based on the dinamically provided filters.
    When filter is ALL it will return all the values.
    """
    with engine.connect() as conn:
        metrics_df = pd.read_sql(query, conn)
    return metrics_df

query_benchmark = text(f"""
    SELECT *
    FROM {"video_nvai_benchmarks"}
    WHERE "platform" LIKE 'dailymotion_ads'
""")

query_metrics = text(f"""
    SELECT *
    FROM {"video_nvai_metrics"}
    WHERE "platform" LIKE 'dailymotion_ads'
""")

run_query(engine, query=query_benchmark ).head()

In [ ]:
run_query(engine, query=query_metrics ).head()